# IMPORTS

In [1]:
# Core data handling libraries
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# The penaltyblog library for data and models
import penaltyblog as pb
from penaltyblog.scrapers import FootballData
from penaltyblog.matchflow import Flow, where_equals, get_field
from penaltyblog.ratings import PiRatingSystem

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from mplsoccer import Pitch

# Modeling and evaluation libraries
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, classification_report

# Suppress Statsbomb's NoAuthWarning
import warnings
from statsbombpy.api_client import NoAuthWarning
warnings.filterwarnings("ignore", category=NoAuthWarning)

In [ ]:
# import os
# temp_dir = os.path.expanduser('~/.cache/v4_temp_data')
# os.makedirs(temp_dir, exist_ok=True)

# # Set the TMPDIR environment variable, which many libraries respect for temp storage
# os.environ['TMPDIR'] = temp_dir

# print(f"Temporary directory for this session has been set to: {os.environ['TMPDIR']}")
# print("You can now safely run the long data aggregation step (Step 3).")

In [2]:
LEAGUE = "ENG Premier League"
SEASON_FD = "2015-2016" # Format for football-data scraper
COMPETITION_ID_SB = 2   # StatsBomb ID for Premier League
SEASON_ID_SB = 27       # StatsBomb ID for 2015/2016 season

# 1. Fetch data from football-data.co.uk (for Odds)
print("Fetching odds data from football-data.co.uk...")
df_odds = FootballData(LEAGUE, SEASON_FD).get_fixtures()
odds_cols = ["date", "team_home", "team_away", "goals_home", "goals_away", "psh", "psd", "psa"]
df_odds = df_odds[odds_cols].dropna(subset=['psh', 'psd', 'psa']).copy()

# 2. Fetch match list from StatsBomb (for Match IDs)
print("Fetching match list from StatsBomb...")
sb_matches_raw = Flow.statsbomb.matches(competition_id=COMPETITION_ID_SB, season_id=SEASON_ID_SB).collect()
df_sb_matches = pd.DataFrame(sb_matches_raw)

# 3. Standardize and Merge
# Extract nested team names from StatsBomb data
df_sb_matches['team_home'] = df_sb_matches['home_team'].apply(lambda x: x['home_team_name'])
df_sb_matches['team_away'] = df_sb_matches['away_team'].apply(lambda x: x['away_team_name'])
df_sb_matches.rename(columns={'match_date': 'date'}, inplace=True)
df_sb_matches = df_sb_matches[['match_id', 'date', 'team_home', 'team_away']]

# Ensure date formats are consistent
df_odds['date'] = pd.to_datetime(df_odds['date'])
df_sb_matches['date'] = pd.to_datetime(df_sb_matches['date'])

# Define mapping for inconsistent team names
name_mapping = {
    'Man United': 'Manchester United', 'Man City': 'Manchester City',
    'West Brom': 'West Bromwich Albion', 'West Ham': 'West Ham United',
    'Stoke': 'Stoke City', 'Swansea': 'Swansea City',
    'Leicester': 'Leicester City', 'Norwich': 'Norwich City',
    'Bournemouth': 'AFC Bournemouth', 'Tottenham': 'Tottenham Hotspur'
}
df_odds.replace({'team_home': name_mapping, 'team_away': name_mapping}, inplace=True)

# Merge the two data sources
print("Merging the two data sources...")
df_master = pd.merge(df_odds, df_sb_matches, on=['date', 'team_home', 'team_away'], how='inner')
df_master.sort_values(by='date', inplace=True)
df_master.reset_index(drop=True, inplace=True)

print(f"\nSuccessfully created master match list with {len(df_master)} matches.")
display(df_master.head())

Fetching odds data from football-data.co.uk...
Fetching match list from StatsBomb...
Merging the two data sources...

Successfully created master match list with 342 matches.


,date,team_home,team_away,goals_home,goals_away,psh,psd,psa,match_id
0,2015-08-08,AFC Bournemouth,Aston Villa,0,1,1.95,3.65,4.27,3754128
1,2015-08-08,Chelsea,Swansea City,2,2,1.39,4.92,10.39,3754078
2,2015-08-08,Everton,Watford,2,2,1.70,3.95,5.62,3754300
3,2015-08-08,Leicester City,Sunderland,4,2,1.99,3.48,4.34,3754237
4,2015-08-08,Manchester United,Tottenham Hotspur,1,0,1.65,4.09,5.90,3754097


In [3]:
all_events_list = []

for match_id in tqdm(df_master['match_id'], desc="Fetching Events for All Matches"):
    try:
        flow = Flow.statsbomb.events(match_id)
        df_match_events = pd.DataFrame(flow.collect())
        df_match_events['match_id'] = match_id
        all_events_list.append(df_match_events)

    except Exception as e:
        print(f"\nSkipping match_id {match_id} due to an error: {e}")
        continue

# Combine all the individual match DataFrames into one large one
print("\nConcatenating all event data...")
df_all_events = pd.concat(all_events_list, ignore_index=True)

print(f"\nProcess complete. Aggregated {len(df_all_events):,} events from {df_master['match_id'].nunique()} matches.")
display(df_all_events.head())

Fetching Events for All Matches:   0%|          | 0/342 [00:00<?, ?it/s]


Concatenating all event data...

Process complete. Aggregated 1,182,698 events from 342 matches.


,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,...,foul_won,shot,goalkeeper,block,substitution,injury_stoppage,bad_behaviour,50_50,half_start,player_off
0,69d801be-03c1-4437-bb6b-812d6e9f6392,1,1,00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 28, 'name': 'AFC Bournemouth'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9d235e73-54b8-4b8f-8cfc-e800336776e4,2,1,00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 28, 'name': 'AFC Bournemouth'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b2eef156-e1e7-43d2-a798-0289762df971,3,1,00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 28, 'name': 'AFC Bournemouth'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,d85c17ec-38c7-464b-bf3c-b2a66f7ece22,4,1,00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 28, 'name': 'AFC Bournemouth'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4708b230-b8af-4854-be26-10806f19c32c,5,1,00:00:00.701,0,0,"{'id': 30, 'name': 'Pass'}",2,"{'id': 59, 'name': 'Aston Villa'}","{'id': 9, 'name': 'From Kick Off'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
all_player_minutes = []

def parse_minutes(time_str):
    if time_str is None:
        return 0
    try:
        m, s = map(int, time_str.split(':'))
        return m + s / 60
    except (ValueError, AttributeError):
        return 0

for match_id in tqdm(df_master['match_id'], desc="Processing Lineups"):
    try:
        # Get the lineup data for the match
        lineups = Flow.statsbomb.lineups(match_id).collect()
        
        for team_lineup in lineups:
            for player in team_lineup['lineup']:
                total_minutes = 0
                for pos_entry in player['positions']:
                    start_min = parse_minutes(pos_entry.get('from', '0:0'))
                    end_min = parse_minutes(pos_entry.get('to'))
                    if end_min == 0:
                        if pos_entry.get('from_period') == 1:
                            end_min = 45 + 5 # End of first half + stoppage
                        elif pos_entry.get('from_period') == 2:
                            end_min = 90 + 10 # End of second half + stoppage
                        else: # Extra time, etc.
                            end_min = start_min + 15

                    total_minutes += (end_min - start_min)

                if total_minutes > 0:
                    all_player_minutes.append({
                        'player_id': player['player_id'],
                        'player_name': player['player_name'],
                        'minutes': total_minutes
                    })

    except Exception as e:
        print(f"\nSkipping lineup for match_id {match_id} due to an error: {e}")
        continue

df_minutes_temp = pd.DataFrame(all_player_minutes)

df_minutes_played = df_minutes_temp.groupby(['player_id', 'player_name'])['minutes'].sum().reset_index()

max_possible_minutes = 38 * 95 
df_minutes_played['minutes'] = df_minutes_played['minutes'].clip(upper=max_possible_minutes)


print(f"\nSuccessfully calculated total minutes for {len(df_minutes_played)} players.")
display(df_minutes_played.sort_values(by='minutes', ascending=False).head(10))

Processing Lineups:   0%|          | 0/342 [00:00<?, ?it/s]


Successfully calculated total minutes for 519 players.


,player_id,player_name,minutes
211,3814,Riyad Mahrez,2643.516667
74,3347,André Ayew Pelé,2432.816667
50,3307,Marc Albrighton,2397.683333
217,3874,Mark Noble,2393.016667
43,3294,Juan Manuel Mata García,2368.316667
119,3496,Mesut Özil,2254.166667
5,3043,Christian Dannemann Eriksen,2241.366667
104,3472,Willian Borges da Silva,2235.583333
248,4275,Ross Barkley,2200.650000
35,3277,Yohan Cabaye,2200.233333


In [11]:
if 'df_all_events' in locals():
    print("--- Columns in df_all_events ---")
    print(sorted(df_all_events.columns.tolist()))

--- Columns in df_all_events ---
['50_50', 'bad_behaviour', 'ball_receipt', 'ball_recovery', 'block', 'carry', 'clearance', 'counterpress', 'dribble', 'duel', 'duration', 'event_type', 'foul_committed', 'foul_won', 'goalkeeper', 'half_start', 'id', 'index', 'injury_stoppage', 'interception', 'location', 'match_id', 'minute', 'miscontrol', 'off_camera', 'out', 'pass', 'period', 'play_pattern', 'player', 'player_id', 'player_name', 'player_off', 'position', 'possession', 'possession_team', 'related_events', 'second', 'shot', 'substitution', 'tactics', 'team', 'team_name', 'timestamp', 'type', 'under_pressure']


In [12]:
print("Building player profiles...")

# --- 1. Prepare the Events DataFrame for easier access ---
df_all_events['player_id'] = df_all_events['player'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
df_all_events['player_name'] = df_all_events['player'].apply(lambda x: x.get('name') if isinstance(x, dict) else None)
df_all_events.dropna(subset=['player_id'], inplace=True)
df_all_events['player_id'] = df_all_events['player_id'].astype(int)
df_all_events['event_type'] = df_all_events['type'].apply(lambda x: x.get('name') if isinstance(x, dict) else '')

# --- 2. Aggregate Key Stats for Each Player ---

# Offensive Stats
df_shots = df_all_events[df_all_events['event_type'] == 'Shot'].copy()
df_shots['xg'] = df_shots['shot'].apply(lambda x: x.get('statsbomb_xg', 0))
player_offense = df_shots.groupby('player_id').agg(
    shots=('id', 'count'),
    xg=('xg', 'sum')
)

# Defensive Stats (using confirmed available events)
player_interceptions = df_all_events[df_all_events['event_type'] == 'Interception'].groupby('player_id').size()
player_pressures = df_all_events[df_all_events['event_type'] == 'Pressure'].groupby('player_id').size()

# --- 3. Combine into a Single Profile DataFrame ---
df_player_profiles = df_minutes_played.copy()
df_player_profiles = pd.merge(df_player_profiles, player_offense, on='player_id', how='left')
df_player_profiles = pd.merge(df_player_profiles, player_interceptions.rename('interceptions'), on='player_id', how='left')
df_player_profiles = pd.merge(df_player_profiles, player_pressures.rename('pressures'), on='player_id', how='left')
df_player_profiles.fillna(0, inplace=True)

# --- 4. Normalize Stats to a Per-90 Minute Basis ---
df_player_profiles['nineties'] = df_player_profiles['minutes'] / 90
stats_to_normalize = ['shots', 'xg', 'interceptions', 'pressures']
for stat in stats_to_normalize:
    df_player_profiles[f'{stat}_p90'] = df_player_profiles[stat] / df_player_profiles['nineties']

df_player_profiles.replace([np.inf, -np.inf], 0, inplace=True)

print("\nPlayer profiles complete.")
display(
    df_player_profiles[df_player_profiles['nineties'] >= 10]
    .sort_values(by='xg_p90', ascending=False)
    .head(10)
)

Building player profiles...

Player profiles complete.


,player_id,player_name,minutes,shots,xg,interceptions,pressures,nineties,shots_p90,xg_p90,interceptions_p90,pressures_p90
382,10955,Harry Kane,2105.816667,152.0,21.388760,11.0,480.0,23.397963,6.496292,0.914129,0.470126,20.514606
385,10960,Jamie Vardy,2182.666667,111.0,21.458020,9.0,462.0,24.251852,4.576970,0.884799,0.371106,19.050092
76,3380,Christian Benteke Liolo,1171.783333,63.0,10.420618,3.0,156.0,13.019815,4.838779,0.800366,0.230418,11.981737
39,3289,Romelu Lukaku Menama,2117.133333,110.0,17.144205,3.0,292.0,23.523704,4.676134,0.728806,0.127531,12.413011
27,3237,Sergio Leonel Agüero del Castillo,2041.116667,108.0,15.343849,8.0,265.0,22.679074,4.762099,0.676564,0.352748,11.684780
290,5458,Odion Jude Ighalo,2092.266667,103.0,15.091118,18.0,467.0,23.247407,4.430602,0.649153,0.774280,20.088262
160,3604,Olivier Giroud,1709.383333,96.0,12.223226,4.0,406.0,18.993148,5.054454,0.643560,0.210602,21.376130
66,3337,Jermain Defoe,1815.983333,83.0,12.896470,4.0,330.0,20.177593,4.113474,0.639148,0.198240,16.354776
288,5198,Diego da Silva Costa,1686.000000,69.0,11.698305,5.0,308.0,18.733333,3.683274,0.624465,0.266904,16.441281
156,3598,Wilfried Guemiand Bony,1193.033333,58.0,7.078127,2.0,95.0,13.255926,4.375402,0.533959,0.150876,7.166606


In [13]:
df_player_profiles.set_index('player_id', inplace=True)

# List to store the feature dictionary for each match
lineup_features_list = []

for _, match in tqdm(df_master.iterrows(), total=df_master.shape[0], desc="Processing Lineups & Features"):
    try:
        match_id = match['match_id']
        home_team_name = match['team_home']
        
        # Fetch lineups for the match
        lineups = Flow.statsbomb.lineups(match_id).collect()

        # Dictionary to hold the summed stats for this match
        match_lineup_stats = {'match_id': match_id}
        
        # The stats we want to aggregate from our player profiles
        stats_to_sum = ['xg_p90', 'shots_p90', 'pressures_p90', 'interceptions_p90']

        # Loop through home and away teams
        for team_lineup in lineups:
            is_home_team = (team_lineup['team_name'] == home_team_name)
            prefix = 'home_starters' if is_home_team else 'away_starters'
            
            # Initialize sums for this team
            summed_stats = {f'{prefix}_{stat}': 0 for stat in stats_to_sum}

            # Get the first 11 players (the starters)
            starting_xi = team_lineup['lineup'][:11]
            
            # Sum the stats for the starting XI
            for player in starting_xi:
                player_id = player['player_id']
                if player_id in df_player_profiles.index:
                    player_profile = df_player_profiles.loc[player_id]
                    for stat in stats_to_sum:
                        summed_stats[f'{prefix}_{stat}'] += player_profile[stat]
            
            # Add the team's summed stats to the match dictionary
            match_lineup_stats.update(summed_stats)
            
        lineup_features_list.append(match_lineup_stats)

    except Exception as e:
        print(f"\nSkipping match_id {match_id} due to an error: {e}")
        continue

df_lineup_features = pd.DataFrame(lineup_features_list)

df_v41_final = pd.merge(df_master, df_lineup_features, on='match_id', how='inner')

print(f"\nLineup-aware feature engineering complete. Final dataset has {df_v41_final.shape[0]} matches.")
display(df_v41_final.head())

Processing Lineups & Features:   0%|          | 0/342 [00:00<?, ?it/s]


Lineup-aware feature engineering complete. Final dataset has 342 matches.


,date,team_home,team_away,goals_home,goals_away,psh,psd,psa,match_id,away_starters_xg_p90,away_starters_shots_p90,away_starters_pressures_p90,away_starters_interceptions_p90,home_starters_xg_p90,home_starters_shots_p90,home_starters_pressures_p90,home_starters_interceptions_p90
0,2015-08-08,AFC Bournemouth,Aston Villa,0,1,1.95,3.65,4.27,3754128,1.000035,13.459312,253.495488,17.161361,1.756263,16.962629,225.918589,15.423498
1,2015-08-08,Chelsea,Swansea City,2,2,1.39,4.92,10.39,3754078,1.282296,14.257589,181.803619,14.653367,1.348009,15.971212,161.528536,12.180472
2,2015-08-08,Everton,Watford,2,2,1.70,3.95,5.62,3754300,1.495270,14.119456,187.839342,14.515919,1.838659,18.957599,201.012081,15.655031
3,2015-08-08,Leicester City,Sunderland,4,2,1.99,3.48,4.34,3754237,1.286905,11.970644,132.491157,10.491877,1.931097,17.533154,232.607745,17.334119
4,2015-08-08,Manchester United,Tottenham Hotspur,1,0,1.65,4.09,5.90,3754097,1.259290,13.920854,210.534281,14.504378,1.257700,13.844591,200.655199,16.182726


In [17]:
# --- 1. Engineer Pi Ratings Probabilities ---
print("Calculating Pi Ratings probabilities for each match...")
pi_ratings = PiRatingSystem()
pi_prob_h, pi_prob_d, pi_prob_a = [], [], []

for _, row in tqdm(df_v41_final.iterrows(), total=df_v41_final.shape[0], desc="Calculating Pi Ratings"):
    home_team = row['team_home']
    away_team = row['team_away']
    
    probabilities = pi_ratings.calculate_match_probabilities(home_team, away_team)
    pi_prob_h.append(probabilities['home_win'])
    pi_prob_d.append(probabilities['draw'])
    pi_prob_a.append(probabilities['away_win'])
    
    goal_diff = row['goals_home'] - row['goals_away']
    pi_ratings.update_ratings(home_team, away_team, goal_diff)

df_v41_final['pi_prob_h'] = pi_prob_h
df_v41_final['pi_prob_d'] = pi_prob_d
df_v41_final['pi_prob_a'] = pi_prob_a


# --- 2. Engineer Market Probabilities ---
print("\nConverting Pinnacle odds to implied probabilities...")
def remove_overround(row):
    odds = [row["psh"], row["psd"], row["psa"]]
    probs = pb.implied.power(odds)["implied_probabilities"]
    return pd.Series(probs)

df_v41_final[['market_prob_h', 'market_prob_d', 'market_prob_a']] = df_v41_final.apply(remove_overround, axis=1)

print("\nFinal feature engineering complete.")

display(df_v41_final[['team_home', 'team_away', 'pi_prob_h', 'market_prob_h']].head())

Calculating Pi Ratings probabilities for each match...


Calculating Pi Ratings:   0%|          | 0/342 [00:00<?, ?it/s]


Converting Pinnacle odds to implied probabilities...

Final feature engineering complete.


,team_home,team_away,pi_prob_h,market_prob_h
0,AFC Bournemouth,Aston Villa,0.308538,0.505858
1,Chelsea,Swansea City,0.308538,0.713645
2,Everton,Watford,0.308538,0.581951
3,Leicester City,Sunderland,0.308538,0.495754
4,Manchester United,Tottenham Hotspur,0.308538,0.599596


In [19]:
# 1. Prepare Data
# Define the Target Variable (y)
def get_result(row):
    if row['goals_home'] > row['goals_away']: return 1
    elif row['goals_home'] < row['goals_away']: return 2
    else: return 0
df_v41_final['result'] = df_v41_final.apply(get_result, axis=1)

# Define our most comprehensive feature set yet
features = [
    # Lineup-Aware Tactical Features
    'home_starters_xg_p90', 'away_starters_xg_p90',
    'home_starters_shots_p90', 'away_starters_shots_p90',
    'home_starters_pressures_p90', 'away_starters_pressures_p90',
    'home_starters_interceptions_p90', 'away_starters_interceptions_p90',
    
    # Pi Ratings (Form Model)
    'pi_prob_h', 'pi_prob_d', 'pi_prob_a',
    
    # Market Features
    'market_prob_h', 'market_prob_d', 'market_prob_a'
]
X = df_v41_final[features]
y = df_v41_final['result']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2. Train and Evaluate the V4.1 Model
print("\nTraining the final V4.1 XGBoost model...")
v41_model = xgb.XGBClassifier(objective='multi:softprob', num_class=3, seed=42)
v41_model.fit(X_train, y_train)
print("Model training complete.")

print("\nEvaluating V4.1 model performance...")
y_pred_proba = v41_model.predict_proba(X_test)
y_pred_class = v41_model.predict(X_test)

loss = log_loss(y_test, y_pred_proba)
accuracy = accuracy_score(y_test, y_pred_class)

print(f"\n--- V4.1 Model Log Loss: {loss:.4f} ---")
print(f"--- V4.1 Model Accuracy: {accuracy:.2%} ---")

print("\nClassification Report:")
target_names = ['Draw', 'Home Win', 'Away Win']
print(classification_report(y_test, y_pred_class, target_names=target_names))


Training the final V4.1 XGBoost model...
Model training complete.

Evaluating V4.1 model performance...

--- V4.1 Model Log Loss: 1.6194 ---
--- V4.1 Model Accuracy: 53.62% ---

Classification Report:
              precision    recall  f1-score   support

        Draw       0.36      0.20      0.26        20
    Home Win       0.55      0.78      0.65        27
    Away Win       0.60      0.55      0.57        22

    accuracy                           0.54        69
   macro avg       0.51      0.51      0.49        69
weighted avg       0.51      0.54      0.51        69



In [21]:
def get_result(row):
    if row['goals_home'] > row['goals_away']: return 1
    elif row['goals_home'] < row['goals_away']: return 2
    else: return 0
df_v41_final['result'] = df_v41_final.apply(get_result, axis=1)
y = df_v41_final['result']

# A helper function to run an experiment
def run_experiment(features, X_df, y_series, experiment_name):
    print(f"\n--- Running Experiment: {experiment_name} ---")
    
    X = X_df[features]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_series, test_size=0.2, random_state=42, stratify=y_series
    )
    
    model = xgb.XGBClassifier(objective='multi:softprob', num_class=3, seed=42)
    model.fit(X_train, y_train)
    
    y_pred_proba = model.predict_proba(X_test)
    y_pred_class = model.predict(X_test)

    loss = log_loss(y_test, y_pred_proba)
    accuracy = accuracy_score(y_test, y_pred_class)

    print(f"Result -> Log Loss: {loss:.4f}, Accuracy: {accuracy:.2%}")
    return loss, accuracy

# --- Experiment 2: Lineup-Aware Stats + Odds ---
features_2 = [
    'home_starters_xg_p90', 'away_starters_xg_p90',
    'home_starters_shots_p90', 'away_starters_shots_p90',
    'home_starters_pressures_p90', 'away_starters_pressures_p90',
    'home_starters_interceptions_p90', 'away_starters_interceptions_p90',
    'market_prob_h', 'market_prob_d', 'market_prob_a'
]
run_experiment(features_2, df_v41_final, y, "Lineup-Aware + Odds")


# --- Experiment 3: Odds + Pi Ratings Only ---
features_3 = [
    'market_prob_h', 'market_prob_d', 'market_prob_a',
    'pi_prob_h', 'pi_prob_d', 'pi_prob_a'
]
run_experiment(features_3, df_v41_final, y, "Odds + Pi Ratings Only")


--- Running Experiment: Lineup-Aware + Odds ---
Result -> Log Loss: 1.7284, Accuracy: 42.03%

--- Running Experiment: Odds + Pi Ratings Only ---
Result -> Log Loss: 1.7435, Accuracy: 46.38%


(1.7435224336453201, 0.463768115942029)